<a href="https://colab.research.google.com/github/carloscesar182/ai_advanced_course/blob/main/Notebooks/RNA/RNAKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
!pip install keras

In [ ]:
# importar os dados
df = pd.read_csv('Churn_treino.csv', sep=';')
df

In [ ]:
# separar x e y
X = df.drop('Exited', axis=1)
y = df['Exited']

In [ ]:
# fazer o labelencoder pra normalizar os dados numéricos
standardscaler = StandardScaler()
numerical = X.select_dtypes(include=['int64', 'float64']).columns
X[numerical] = standardscaler.fit_transform(X[numerical])
X

In [ ]:
# fazer o labelencoder pra transformar os dados categóricos
# importante, se a classe não fosse numérica, teríamos que aplicar o labelencoder nela tbm pq em redes neurais é necessário
labelencoder = LabelEncoder()
categorical = X.select_dtypes(include=['object']).columns
for col in categorical:
    X[col] = labelencoder.fit_transform(X[col])
X

In [ ]:
# dividir treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# montar a topologia da rede neural
modelo = Sequential()
# adicionar as camadas
# tipo densa, unitis é o num de neuronios, função de ativação, input_dim é o num de elementos da nossa camada
modelo.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
modelo.add(Dropout(0.4)) # regularização pra reduzir overfitting
modelo.add(Dense(units=32, activation='relu'))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=64, activation='relu'))
modelo.add(Dropout(0.4))
# se eu to prevendo 0 ou 1, preciso apenas de 1 neuronio (units=1) e a função sigmoid
modelo.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# compilar e treinar o modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelo.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# fazer previsões
previsoes = modelo.predict(X_test)
previsoes = (previsoes > 0.5).astype('int32')
previsoes

In [ ]:
# calcular e mostrar as metricas
print('Acurácia: ', accuracy_score(y_test, previsoes))
print('F1 Score: ', f1_score(y_test, previsoes))
print('Recall: ', recall_score(y_test, previsoes))
print('Matriz de confusão: \n', confusion_matrix(y_test, previsoes))